# Simulation 01 -- Baseline Recovery Test

## Question
**Under ideal conditions (symmetric distortions, good features), can the pipeline recover the true FMV?**

## Approach
We run the full 4-stage pipeline on the synthetic 5,000-site dataset where we *know* the ground truth FMV. The synthetic data was generated with:
- ~10% origination noise (symmetric, log-normal)
- Contractual escalation over 2–25 year leases
- No systematic bias (no carrier-specific premium, no asymmetric property-type effects)

If the pipeline cannot recover FMV here, it will certainly fail on real data.

### Pipeline Stages
| Stage | Description |
|-------|------------|
| **1** | HDBSCAN clustering on market features (X_MARKET) |
| **2** | MAD-based pseudo-labeling within each cluster |
| **3** | XGBoost + Quantile Random Forest trained on CORE sites |
| **4** | Predict FMV for all sites and classify OVERPAYING / AT_MARKET / UNDERPAYING |

### GO / NO-GO Criteria
- MAPE < 15%
- Classification accuracy > 70%

In [ ]:
# ============================================================
# Imports & Configuration
# ============================================================
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import median_abs_deviation
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    confusion_matrix, classification_report, accuracy_score,
    silhouette_score,
)

from xgboost import XGBRegressor

# Quantile forest -- graceful fallback
try:
    from quantile_forest import RandomForestQuantileRegressor
    HAS_QF = True
    print('\u2713 quantile-forest available -- using RandomForestQuantileRegressor')
except ImportError:
    HAS_QF = False
    print('\u2717 quantile-forest not installed -- falling back to sklearn RF with manual quantile estimation')

# HDBSCAN -- try sklearn >= 1.3, then hdbscan library
try:
    from sklearn.cluster import HDBSCAN as SklearnHDBSCAN
    HAS_SKLEARN_HDBSCAN = True
    print('\u2713 Using sklearn.cluster.HDBSCAN')
except ImportError:
    HAS_SKLEARN_HDBSCAN = False
    import hdbscan
    print('\u2713 Using hdbscan library')

np.random.seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 110

print('\nAll imports loaded successfully.')

---
## Stage 0: Load Synthetic Data & Define Feature Sets

We load the synthetic dataset produced by `sim_00_data_generator.ipynb`. This file
contains 5,000 cell-tower sites with:
- Market/demographic features (census data, land values, zoning)
- Network/carrier features (tenant count, technology, backhaul)
- Lease parameters (escalation type, lease age)
- **Ground truth FMV** (`true_fmv`) for evaluation
- **Distorted current rents** (`current_monthly_rent`) that simulate real-world contractual drift

In [ ]:
# ============================================================
# Load data
# ============================================================
df = pd.read_parquet('../data/synthetic/synthetic_5k_sites.parquet')
print(f'Loaded {len(df):,} sites  |  {len(df.columns)} columns')
print(f'True FMV range: ${df["true_fmv"].min():,.0f} -- ${df["true_fmv"].max():,.0f}')
print(f'Current rent range: ${df["current_monthly_rent"].min():,.0f} -- ${df["current_monthly_rent"].max():,.0f}')
print(f'\nCorrelation (true_fmv vs current_rent): {df["true_fmv"].corr(df["current_monthly_rent"]):.4f}')
df.head(3)

In [ ]:
# ============================================================
# Define feature lists
# ============================================================

# X_MARKET: features available for ALL sites (clustering + supervised).
# These capture location value, property characteristics, competition,
# zoning/regulatory environment -- everything EXCEPT rent and network params.
X_MARKET = [
    # Demographics & Market (Domain 5)
    'census_population_density', 'census_population_3km',
    'census_median_income', 'census_median_home_value',
    'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf',
    'hud_safmr', 'property_tax_rate',
    # Topography / Geography
    'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m',
    'distance_to_highway_km', 'ground_elevation_ft',
    'unemployment_rate_local',
    # Competition / Supply (Domain 7)
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
    'competitor_tower_distance_km', 'tower_per_capita',
    'scarcity_index', 'landlord_concentration',
    'composite_hazard_score',
    # Property / Structure (Domain 1 & 3)
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage',
    'construction_age', 'site_access_ordinal',
    # Zoning / Regulatory (Domain 2)
    'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]

# Z_NETWORK: carrier/network features (only for supervised model, not clustering)
Z_NETWORK = [
    'tenant_count', 'primary_technology', 'coverage_critical',
    'capacity_utilization_pct', 'high_traffic', 'backhaul_type',
    'has_fiber_connection', 'network_substitutability_score',
    'search_ring_alternative_count', 'inter_site_distance_km',
    'antenna_count', 'structure_capacity_remaining',
    'site_priority_tier',
]

# Full feature set for supervised model = X_MARKET + Z_NETWORK
ALL_FEATURES = X_MARKET + Z_NETWORK

# Categorical columns that need encoding
CAT_ENCODE = {
    'property_type': 'onehot',           # 7 distinct property types
    'structure_type': 'label',           # ordinal-ish categories
    'zoning_district_type': 'label',     # ordinal-ish categories
}

print(f'X_MARKET features:   {len(X_MARKET)}')
print(f'Z_NETWORK features:  {len(Z_NETWORK)}')
print(f'ALL_FEATURES:        {len(ALL_FEATURES)}')
print(f'Categorical to encode: {list(CAT_ENCODE.keys())}')

In [ ]:
# ============================================================
# Encode categorical features
# ============================================================
df_model = df.copy()

# Label-encode structure_type and zoning_district_type
le_structure = LabelEncoder()
df_model['structure_type_enc'] = le_structure.fit_transform(df_model['structure_type'])

le_zoning = LabelEncoder()
df_model['zoning_district_type_enc'] = le_zoning.fit_transform(df_model['zoning_district_type'])

# One-hot encode property_type
prop_dummies = pd.get_dummies(df_model['property_type'], prefix='pt')
df_model = pd.concat([df_model, prop_dummies], axis=1)

# Build augmented feature lists that include the encoded columns
ENCODED_CATS = ['structure_type_enc', 'zoning_district_type_enc'] + list(prop_dummies.columns)

X_MARKET_ENC = X_MARKET + ENCODED_CATS
ALL_FEATURES_ENC = ALL_FEATURES + ENCODED_CATS

print(f'Encoded feature set sizes:')
print(f'  X_MARKET_ENC:     {len(X_MARKET_ENC)}')
print(f'  ALL_FEATURES_ENC: {len(ALL_FEATURES_ENC)}')
print(f'  Property type dummies: {list(prop_dummies.columns)}')
print(f'  Label encodings:')
print(f'    structure_type       -> {list(le_structure.classes_)}')
print(f'    zoning_district_type -> {list(le_zoning.classes_)}')

---
## Stage 1: HDBSCAN Clustering on Market Features

We cluster sites on **X_MARKET** features only (no rent, no Z_NETWORK). The goal is to group
sites into "comparable" market tiers so that within each cluster, rents should be relatively
homogeneous -- allowing the MAD filter in Stage 2 to identify outliers effectively.

Key decisions:
- **StandardScaler** on all features (HDBSCAN is distance-based; features must be on comparable scales)
- `min_cluster_size=20` -- small enough for fine-grained tiers, large enough for stable statistics
- No rent features enter clustering -- this avoids circular reasoning

In [ ]:
# ============================================================
# Stage 1: HDBSCAN Clustering
# ============================================================

# Scale continuous features for distance-based clustering
scaler_clust = StandardScaler()
X_clust = scaler_clust.fit_transform(df_model[X_MARKET_ENC].fillna(0))

# Fit HDBSCAN
if HAS_SKLEARN_HDBSCAN:
    clusterer = SklearnHDBSCAN(
        min_cluster_size=20,
        min_samples=5,
        metric='euclidean',
        n_jobs=-1,
    )
else:
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=20,
        min_samples=5,
        metric='euclidean',
        core_dist_n_jobs=-1,
    )

df_model['cluster'] = clusterer.fit_predict(X_clust)

n_clusters = df_model['cluster'].nunique() - (1 if -1 in df_model['cluster'].values else 0)
n_noise = (df_model['cluster'] == -1).sum()

print(f'HDBSCAN Results:')
print(f'  Clusters found:  {n_clusters}')
print(f'  Noise points:    {n_noise} ({n_noise / len(df_model) * 100:.1f}%)')
print(f'\nCluster size distribution (top 15):')
print(df_model['cluster'].value_counts().head(15))

In [ ]:
# ============================================================
# Silhouette score (excluding noise points)
# ============================================================
non_noise_mask = df_model['cluster'] != -1

if non_noise_mask.sum() > 100 and n_clusters >= 2:
    sil_score = silhouette_score(
        X_clust[non_noise_mask],
        df_model.loc[non_noise_mask, 'cluster'],
        sample_size=min(5000, non_noise_mask.sum()),
        random_state=42,
    )
    print(f'Silhouette score (excl. noise): {sil_score:.3f}')
else:
    sil_score = np.nan
    print('Silhouette score: N/A (too few clusters or too few non-noise points)')

In [ ]:
# ============================================================
# Compare within-cluster rent CV vs portfolio-wide CV
# ============================================================
# Coefficient of Variation = std / mean  (lower = more homogeneous)

portfolio_cv = df_model['current_monthly_rent'].std() / df_model['current_monthly_rent'].mean()

cluster_cvs = []
for cid, grp in df_model[non_noise_mask].groupby('cluster'):
    if len(grp) >= 5:
        cv = grp['current_monthly_rent'].std() / grp['current_monthly_rent'].mean()
        cluster_cvs.append({'cluster': cid, 'n': len(grp), 'cv': cv})

cv_df = pd.DataFrame(cluster_cvs)
mean_within_cv = cv_df['cv'].mean()
median_within_cv = cv_df['cv'].median()

print(f'Portfolio-wide rent CV:   {portfolio_cv:.3f}')
print(f'Mean within-cluster CV:   {mean_within_cv:.3f}')
print(f'Median within-cluster CV: {median_within_cv:.3f}')
print(f'CV reduction ratio:       {mean_within_cv / portfolio_cv:.2f}x  '
      f'({(1 - mean_within_cv / portfolio_cv) * 100:.0f}% reduction)')

# -- Plot --
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.hist(cv_df['cv'], bins=30, color='steelblue', edgecolor='white', alpha=0.8)
ax.axvline(portfolio_cv, color='red', ls='--', lw=2,
           label=f'Portfolio CV = {portfolio_cv:.3f}')
ax.axvline(mean_within_cv, color='orange', ls='--', lw=2,
           label=f'Mean within-cluster CV = {mean_within_cv:.3f}')
ax.set_xlabel('Coefficient of Variation (rent)')
ax.set_ylabel('Number of clusters')
ax.set_title('Stage 1: Within-Cluster Rent Homogeneity')
ax.legend()
plt.tight_layout()
plt.show()

---
## Stage 2: Pseudo-Labeling with MAD Threshold

Within each cluster, we compute the **Median Absolute Deviation (MAD)** of `current_monthly_rent`
and assign labels based on the z-score (distance from median in MAD units):

| Label | Condition | Interpretation |
|-------|-----------|----------------|
| **CORE** | z-score < 1.0 | Rents close to cluster median -- likely near true FMV |
| **MARGINAL** | 1.0 \u2264 z-score < 2.0 | Moderately distorted -- excluded from training |
| **OUTLIER** | z-score \u2265 2.0 | Highly distorted -- definitely excluded |

CORE sites become the training set for the supervised model. The assumption is that
under symmetric noise, the cluster median is a good proxy for FMV, so sites close to that
median are the most reliable pseudo-labels.

In [ ]:
# ============================================================
# Stage 2: MAD-based pseudo-labeling
# ============================================================
MAD_THRESHOLD = 1.0
OUTLIER_THRESHOLD = 2.0


def pseudo_label_clusters(df_in, mad_thresh=MAD_THRESHOLD, outlier_thresh=OUTLIER_THRESHOLD):
    """
    For each cluster, compute median rent and MAD, then assign labels.
    Noise points (cluster == -1) are labeled MARGINAL by default.
    
    Returns
    -------
    labels : pd.Series of str
    z_scores : pd.Series of float
    """
    labels = pd.Series('MARGINAL', index=df_in.index)
    z_scores = pd.Series(np.nan, index=df_in.index)

    for cid, grp in df_in.groupby('cluster'):
        if cid == -1:
            continue  # noise points stay MARGINAL

        rents = grp['current_monthly_rent']
        med = rents.median()
        mad = median_abs_deviation(rents, scale='normal')  # scaled to std-equiv

        if mad < 1e-6:  # avoid division by zero for perfectly uniform clusters
            mad = rents.std() if rents.std() > 0 else 1.0

        z = np.abs(rents - med) / mad
        z_scores.loc[grp.index] = z

        labels.loc[grp.index[z < mad_thresh]] = 'CORE'
        labels.loc[grp.index[(z >= mad_thresh) & (z < outlier_thresh)]] = 'MARGINAL'
        labels.loc[grp.index[z >= outlier_thresh]] = 'OUTLIER'

    return labels, z_scores


df_model['pseudo_label'], df_model['mad_z_score'] = pseudo_label_clusters(df_model)

# -- Distribution --
label_counts = df_model['pseudo_label'].value_counts()
print('Pseudo-label distribution:')
for lab in ['CORE', 'MARGINAL', 'OUTLIER']:
    n = label_counts.get(lab, 0)
    print(f'  {lab:10s}: {n:5d}  ({n / len(df_model) * 100:5.1f}%)')

core_pct = label_counts.get('CORE', 0) / len(df_model)
print(f'\nCore coverage: {core_pct:.1%}')
assert core_pct > 0.30, f'WARN: Core coverage too low ({core_pct:.1%}). Pipeline may be unreliable.'

In [ ]:
# ============================================================
# Check coverage: property type and state
# ============================================================
core_mask = df_model['pseudo_label'] == 'CORE'

# Property-type coverage
prop_coverage = df_model.loc[core_mask, 'property_type'].value_counts()
prop_total = df_model['property_type'].value_counts()

print('Property type coverage in CORE set:')
for pt in prop_total.index:
    n_core = prop_coverage.get(pt, 0)
    n_total = prop_total[pt]
    print(f'  {pt:15s}: {n_core:4d} / {n_total:4d}  ({n_core / n_total * 100:5.1f}%)')

# State coverage
state_coverage = df_model.loc[core_mask, 'state'].nunique()
state_total = df_model['state'].nunique()
print(f'\nState coverage: {state_coverage} / {state_total} states represented in CORE')

---
## Stage 3: Supervised Model -- XGBoost + QRF with Ridge Meta-Learner

We train on **CORE** sites only, using **ALL_FEATURES** (X_MARKET + Z_NETWORK) to predict
`current_monthly_rent` (which for CORE sites is our pseudo-FMV).

**Spatial CV** with `GroupKFold` on `state` ensures no geographic leakage between folds.

### Architecture
- **XGBRegressor**: Gradient-boosted point predictions (n_estimators=500, lr=0.05)
- **QRF** (RandomForestQuantileRegressor or fallback): Quantile predictions at [10th, 25th, 50th, 75th, 90th] percentiles
- **Ridge meta-learner**: Combines XGB + QRF-median out-of-fold predictions into a final FMV estimate

In [ ]:
# ============================================================
# Stage 3: Build training data
# ============================================================
core_df = df_model[core_mask].copy()
print(f'Training on {len(core_df):,} CORE sites')

X_train_all = core_df[ALL_FEATURES_ENC].fillna(0).values
y_train_all = core_df['current_monthly_rent'].values
groups_all = core_df['state'].values

print(f'Feature matrix shape: {X_train_all.shape}')
print(f'Target range: ${y_train_all.min():,.0f} -- ${y_train_all.max():,.0f}')
print(f'Unique groups (states): {len(np.unique(groups_all))}')

In [ ]:
# ============================================================
# Stage 3: GroupKFold Cross-Validation
# ============================================================
N_SPLITS = 5
gkf = GroupKFold(n_splits=N_SPLITS)

# Arrays to store out-of-fold predictions
oof_xgb = np.full(len(core_df), np.nan)
oof_qrf_median = np.full(len(core_df), np.nan)
oof_qrf_q10 = np.full(len(core_df), np.nan)
oof_qrf_q25 = np.full(len(core_df), np.nan)
oof_qrf_q75 = np.full(len(core_df), np.nan)
oof_qrf_q90 = np.full(len(core_df), np.nan)

# Trained models for full-dataset prediction
xgb_models = []
qrf_models = []

fold_metrics = []

for fold_i, (train_idx, val_idx) in enumerate(gkf.split(X_train_all, y_train_all, groups_all)):
    X_tr, X_val = X_train_all[train_idx], X_train_all[val_idx]
    y_tr, y_val = y_train_all[train_idx], y_train_all[val_idx]

    # --- XGBoost ---
    xgb_model = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=5,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42 + fold_i,
        n_jobs=-1,
        verbosity=0,
    )
    xgb_model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False,
    )
    oof_xgb[val_idx] = xgb_model.predict(X_val)
    xgb_models.append(xgb_model)

    # --- Quantile Random Forest ---
    if HAS_QF:
        qrf_model = RandomForestQuantileRegressor(
            n_estimators=500,
            min_samples_leaf=5,
            random_state=42 + fold_i,
            n_jobs=-1,
        )
        qrf_model.fit(X_tr, y_tr)

        quantile_preds = qrf_model.predict(
            X_val, quantiles=[0.10, 0.25, 0.50, 0.75, 0.90]
        )
        oof_qrf_q10[val_idx] = quantile_preds[:, 0]
        oof_qrf_q25[val_idx] = quantile_preds[:, 1]
        oof_qrf_median[val_idx] = quantile_preds[:, 2]
        oof_qrf_q75[val_idx] = quantile_preds[:, 3]
        oof_qrf_q90[val_idx] = quantile_preds[:, 4]
    else:
        # Fallback: standard RF with manual quantile estimation from tree predictions
        qrf_model = RandomForestRegressor(
            n_estimators=500,
            min_samples_leaf=5,
            random_state=42 + fold_i,
            n_jobs=-1,
        )
        qrf_model.fit(X_tr, y_tr)

        tree_preds = np.array([tree.predict(X_val) for tree in qrf_model.estimators_])
        oof_qrf_q10[val_idx] = np.percentile(tree_preds, 10, axis=0)
        oof_qrf_q25[val_idx] = np.percentile(tree_preds, 25, axis=0)
        oof_qrf_median[val_idx] = np.percentile(tree_preds, 50, axis=0)
        oof_qrf_q75[val_idx] = np.percentile(tree_preds, 75, axis=0)
        oof_qrf_q90[val_idx] = np.percentile(tree_preds, 90, axis=0)

    qrf_models.append(qrf_model)

    # Fold metrics
    fold_mae = mean_absolute_error(y_val, oof_xgb[val_idx])
    fold_mape = np.mean(np.abs((y_val - oof_xgb[val_idx]) / y_val)) * 100
    fold_r2 = r2_score(y_val, oof_xgb[val_idx])
    fold_metrics.append({
        'fold': fold_i + 1, 'mae': fold_mae, 'mape': fold_mape, 'r2': fold_r2,
        'train_n': len(train_idx), 'val_n': len(val_idx),
    })
    print(f'  Fold {fold_i + 1}: MAE=${fold_mae:,.0f}  MAPE={fold_mape:.1f}%  '
          f'R\u00b2={fold_r2:.3f}  (train={len(train_idx)}, val={len(val_idx)})')

fold_df = pd.DataFrame(fold_metrics)
print(f'\nOOF Summary (XGB):')
print(f'  Mean MAE:  ${fold_df["mae"].mean():,.0f}')
print(f'  Mean MAPE: {fold_df["mape"].mean():.1f}%')
print(f'  Mean R\u00b2:   {fold_df["r2"].mean():.3f}')

In [ ]:
# ============================================================
# Stage 3: Ridge Meta-Learner on OOF predictions
# ============================================================

# Stack XGB and QRF-median predictions
valid_oof_mask = ~np.isnan(oof_xgb) & ~np.isnan(oof_qrf_median)
X_meta = np.column_stack([oof_xgb[valid_oof_mask], oof_qrf_median[valid_oof_mask]])
y_meta = y_train_all[valid_oof_mask]

meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta, y_meta)

print(f'Ridge meta-learner coefficients:')
print(f'  XGB weight:        {meta_model.coef_[0]:.4f}')
print(f'  QRF median weight: {meta_model.coef_[1]:.4f}')
print(f'  Intercept:         {meta_model.intercept_:.2f}')

# OOF meta predictions on the CORE set
oof_meta = meta_model.predict(X_meta)
oof_meta_mae = mean_absolute_error(y_meta, oof_meta)
oof_meta_mape = np.mean(np.abs((y_meta - oof_meta) / y_meta)) * 100
oof_meta_r2 = r2_score(y_meta, oof_meta)

print(f'\nMeta-learner OOF performance (CORE sites):')
print(f'  MAE:  ${oof_meta_mae:,.0f}')
print(f'  MAPE: {oof_meta_mape:.1f}%')
print(f'  R\u00b2:   {oof_meta_r2:.3f}')

---
## Stage 4: Predict FMV for ALL Sites & Classify

Now we predict on **all 5,000 sites** (not just CORE) using the ensemble of fold models,
then classify each site as OVERPAYING, AT_MARKET, or UNDERPAYING based on QRF quantile bands.

Classification rule:
- **OVERPAYING**: current rent > QRF Q75 of the predicted FMV distribution
- **UNDERPAYING**: current rent < QRF Q25
- **AT_MARKET**: otherwise

In [ ]:
# ============================================================
# Stage 4: Full-dataset prediction
# ============================================================
X_all = df_model[ALL_FEATURES_ENC].fillna(0).values

# Average XGB predictions across fold models
xgb_preds_all = np.mean([m.predict(X_all) for m in xgb_models], axis=0)

# Average QRF quantile predictions across fold models
if HAS_QF:
    qrf_preds_list = []
    for m in qrf_models:
        qrf_preds_list.append(
            m.predict(X_all, quantiles=[0.10, 0.25, 0.50, 0.75, 0.90])
        )
    qrf_preds_all = np.mean(qrf_preds_list, axis=0)
    qrf_q10_all = qrf_preds_all[:, 0]
    qrf_q25_all = qrf_preds_all[:, 1]
    qrf_median_all = qrf_preds_all[:, 2]
    qrf_q75_all = qrf_preds_all[:, 3]
    qrf_q90_all = qrf_preds_all[:, 4]
else:
    # Manual quantile estimation from all tree predictions across all folds
    all_tree_preds = []
    for m in qrf_models:
        for tree in m.estimators_:
            all_tree_preds.append(tree.predict(X_all))
    all_tree_preds = np.array(all_tree_preds)
    qrf_q10_all = np.percentile(all_tree_preds, 10, axis=0)
    qrf_q25_all = np.percentile(all_tree_preds, 25, axis=0)
    qrf_median_all = np.percentile(all_tree_preds, 50, axis=0)
    qrf_q75_all = np.percentile(all_tree_preds, 75, axis=0)
    qrf_q90_all = np.percentile(all_tree_preds, 90, axis=0)

# Meta-learner final prediction
X_meta_all = np.column_stack([xgb_preds_all, qrf_median_all])
predicted_fmv = meta_model.predict(X_meta_all)

# Clip to reasonable range
predicted_fmv = np.clip(predicted_fmv, 100, 15000)

# Store results
df_model['predicted_fmv'] = predicted_fmv
df_model['pred_q10'] = qrf_q10_all
df_model['pred_q25'] = qrf_q25_all
df_model['pred_q50'] = qrf_median_all
df_model['pred_q75'] = qrf_q75_all
df_model['pred_q90'] = qrf_q90_all

print(f'Predicted FMV distribution (all {len(df_model):,} sites):')
print(f'  Min:    ${predicted_fmv.min():,.0f}')
print(f'  Q25:    ${np.percentile(predicted_fmv, 25):,.0f}')
print(f'  Median: ${np.median(predicted_fmv):,.0f}')
print(f'  Q75:    ${np.percentile(predicted_fmv, 75):,.0f}')
print(f'  Max:    ${predicted_fmv.max():,.0f}')

In [ ]:
# ============================================================
# Stage 4: Classify sites
# ============================================================
rent = df_model['current_monthly_rent'].values

model_class = np.where(
    rent > qrf_q75_all, 'OVERPAYING',
    np.where(rent < qrf_q25_all, 'UNDERPAYING', 'AT_MARKET')
)
df_model['model_class'] = model_class

print('Model classification distribution:')
for cls in ['OVERPAYING', 'AT_MARKET', 'UNDERPAYING']:
    n = (model_class == cls).sum()
    print(f'  {cls:12s}: {n:5d}  ({n / len(df_model) * 100:5.1f}%)')

---
## Evaluation Against Ground Truth

Now we compare our predictions to the **true FMV** that was generated in Simulation 00.
This is the definitive test: if the pipeline can't recover known FMV under ideal conditions,
it won't work on real data.

We evaluate both:
1. **Regression quality**: How close is `predicted_fmv` to `true_fmv`? (MAE, MAPE, RMSE, R\u00b2)
2. **Classification quality**: Does the model correctly identify which sites are OVERPAYING, AT_MARKET, or UNDERPAYING? (confusion matrix, precision, recall, F1)

In [ ]:
# ============================================================
# Regression metrics: predicted FMV vs true FMV
# ============================================================
y_true = df_model['true_fmv'].values
y_pred = df_model['predicted_fmv'].values

mae = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)
pearson_r, pearson_p = stats.pearsonr(y_true, y_pred)
spearman_r, spearman_p = stats.spearmanr(y_true, y_pred)

print('=' * 60)
print('REGRESSION METRICS: Predicted FMV vs True FMV')
print('=' * 60)
print(f'  MAE:             ${mae:,.0f}/month')
print(f'  MAPE:            {mape:.2f}%')
print(f'  RMSE:            ${rmse:,.0f}/month')
print(f'  R-squared:       {r2:.4f}')
print(f'  Pearson r:       {pearson_r:.4f}  (p={pearson_p:.2e})')
print(f'  Spearman rho:    {spearman_r:.4f}  (p={spearman_p:.2e})')

In [ ]:
# ============================================================
# Classification metrics: model class vs true class
# ============================================================
true_class = df_model['true_class'].values
pred_class = df_model['model_class'].values

class_labels = ['UNDERPAYING', 'AT_MARKET', 'OVERPAYING']

acc = accuracy_score(true_class, pred_class)
cm = confusion_matrix(true_class, pred_class, labels=class_labels)
report = classification_report(true_class, pred_class, labels=class_labels, digits=3)

print('=' * 60)
print('CLASSIFICATION METRICS: Model Class vs True Class')
print('=' * 60)
print(f'  Overall Accuracy: {acc:.3f} ({acc * 100:.1f}%)')
print(f'\nConfusion Matrix (rows=true, cols=predicted):')
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)
print(cm_df.to_string())
print(f'\nPer-class Report:')
print(report)

In [ ]:
# ============================================================
# Plot 1: Scatter -- Predicted FMV vs True FMV
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: colored by true class
colors_map = {'OVERPAYING': '#e74c3c', 'AT_MARKET': '#2ecc71', 'UNDERPAYING': '#3498db'}
for cls in class_labels:
    mask = true_class == cls
    axes[0].scatter(y_true[mask], y_pred[mask], alpha=0.3, s=8,
                    c=colors_map[cls], label=cls)

lims = [min(y_true.min(), y_pred.min()) * 0.9,
        max(y_true.max(), y_pred.max()) * 1.05]
axes[0].plot(lims, lims, 'k--', lw=1, alpha=0.7, label='Perfect prediction')
axes[0].set_xlabel('True FMV ($/month)', fontsize=11)
axes[0].set_ylabel('Predicted FMV ($/month)', fontsize=11)
axes[0].set_title(f'Predicted vs True FMV  (MAPE={mape:.1f}%, R\u00b2={r2:.3f})', fontsize=12)
axes[0].legend(fontsize=9)
axes[0].set_xlim(lims)
axes[0].set_ylim(lims)

# Right: density heatmap
axes[1].hexbin(y_true, y_pred, gridsize=50, cmap='YlOrRd', mincnt=1)
axes[1].plot(lims, lims, 'k--', lw=1, alpha=0.7)
axes[1].set_xlabel('True FMV ($/month)', fontsize=11)
axes[1].set_ylabel('Predicted FMV ($/month)', fontsize=11)
axes[1].set_title('Density Heatmap', fontsize=12)
plt.colorbar(axes[1].collections[0], ax=axes[1], label='Count')
axes[1].set_xlim(lims)
axes[1].set_ylim(lims)

plt.suptitle('Simulation 01: Baseline Recovery -- FMV Prediction Quality',
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Plot 2: Error Distribution Histogram
# ============================================================
errors = y_pred - y_true
pct_errors = (y_pred - y_true) / y_true * 100

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Absolute errors
axes[0].hist(errors, bins=80, color='steelblue', edgecolor='white', alpha=0.8)
axes[0].axvline(0, color='red', ls='--', lw=1.5)
axes[0].axvline(errors.mean(), color='orange', ls='--', lw=1.5,
                label=f'Mean error = ${errors.mean():,.0f}')
axes[0].set_xlabel('Prediction Error ($/month)', fontsize=11)
axes[0].set_ylabel('Count', fontsize=11)
axes[0].set_title('Absolute Error Distribution', fontsize=12)
axes[0].legend()

# Percentage errors
axes[1].hist(pct_errors, bins=80, color='coral', edgecolor='white', alpha=0.8)
axes[1].axvline(0, color='red', ls='--', lw=1.5)
axes[1].axvline(pct_errors.mean(), color='orange', ls='--', lw=1.5,
                label=f'Mean = {pct_errors.mean():.1f}%')
axes[1].set_xlabel('Prediction Error (%)', fontsize=11)
axes[1].set_ylabel('Count', fontsize=11)
axes[1].set_title('Percentage Error Distribution', fontsize=12)
axes[1].legend()

plt.suptitle('Simulation 01: Error Distribution', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

print(f'Error statistics:')
print(f'  Mean error:   ${errors.mean():,.0f} ({pct_errors.mean():.1f}%)')
print(f'  Median error: ${np.median(errors):,.0f} ({np.median(pct_errors):.1f}%)')
print(f'  Std error:    ${errors.std():,.0f} ({pct_errors.std():.1f}%)')

In [ ]:
# ============================================================
# Plot 3: Error by Property Type
# ============================================================
df_model['abs_pct_error'] = np.abs(pct_errors)
df_model['signed_pct_error'] = pct_errors

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Absolute percentage error by property type
prop_order = (df_model.groupby('property_type')['abs_pct_error']
              .median().sort_values().index)
sns.boxplot(data=df_model, x='property_type', y='abs_pct_error', order=prop_order,
            palette='Set2', ax=axes[0], showfliers=False)
axes[0].set_xlabel('Property Type', fontsize=11)
axes[0].set_ylabel('|Prediction Error| (%)', fontsize=11)
axes[0].set_title('Absolute Error by Property Type', fontsize=12)
axes[0].tick_params(axis='x', rotation=30)

# Signed error by property type (check for systematic bias)
sns.boxplot(data=df_model, x='property_type', y='signed_pct_error', order=prop_order,
            palette='Set2', ax=axes[1], showfliers=False)
axes[1].axhline(0, color='red', ls='--', lw=1)
axes[1].set_xlabel('Property Type', fontsize=11)
axes[1].set_ylabel('Signed Error (%)', fontsize=11)
axes[1].set_title('Signed Error by Property Type (bias check)', fontsize=12)
axes[1].tick_params(axis='x', rotation=30)

plt.suptitle('Simulation 01: Error Analysis by Property Type', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

# Summary table
pt_summary = df_model.groupby('property_type').agg(
    n=('true_fmv', 'size'),
    median_abs_error=('abs_pct_error', 'median'),
    mean_signed_error=('signed_pct_error', 'mean'),
).round(2).sort_values('median_abs_error')
print('\nError by property type:')
print(pt_summary.to_string())

In [ ]:
# ============================================================
# Plot 4: Confusion matrix heatmap
# ============================================================
fig, ax = plt.subplots(1, 1, figsize=(7, 5.5))

cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels, ax=ax,
            cbar_kws={'label': 'Proportion'})

# Add raw counts as secondary annotation
for i in range(len(class_labels)):
    for j in range(len(class_labels)):
        ax.text(j + 0.5, i + 0.72, f'(n={cm[i, j]})',
                ha='center', va='center', fontsize=8, color='gray')

ax.set_xlabel('Predicted Class', fontsize=11)
ax.set_ylabel('True Class', fontsize=11)
ax.set_title(f'Classification Confusion Matrix  (Accuracy={acc:.1%})', fontsize=12)
plt.tight_layout()
plt.show()

---
## GO / NO-GO Decision

The two criteria that must be met for the pipeline to pass the baseline recovery test:

1. **MAPE < 15%** -- the average percentage error of the predicted FMV must be below 15%
2. **Classification accuracy > 70%** -- at least 70% of sites must be correctly classified into OVERPAYING / AT_MARKET / UNDERPAYING

If both pass, the pipeline is considered viable for deployment on real Verizon data.

In [ ]:
# ============================================================
# GO / NO-GO Decision
# ============================================================
MAPE_THRESHOLD = 15.0
ACC_THRESHOLD = 0.70

mape_pass = mape < MAPE_THRESHOLD
acc_pass = acc > ACC_THRESHOLD

print('=' * 60)
print('SIMULATION 01: GO / NO-GO DECISION')
print('=' * 60)
print(f'\n  Criterion 1 -- MAPE < {MAPE_THRESHOLD}%:')
print(f'    Achieved: {mape:.2f}%')
print(f'    Result:   {"PASS" if mape_pass else "FAIL"}')

print(f'\n  Criterion 2 -- Classification Accuracy > {ACC_THRESHOLD:.0%}:')
print(f'    Achieved: {acc:.1%}')
print(f'    Result:   {"PASS" if acc_pass else "FAIL"}')

overall = mape_pass and acc_pass
print(f'\n  {"=" * 40}')
if overall:
    print(f'  OVERALL: GO -- Pipeline passes baseline recovery test')
else:
    print(f'  OVERALL: NO-GO -- Pipeline fails baseline recovery test')
print(f'  {"=" * 40}')

print(f'\nKey metrics summary:')
print(f'  MAPE:             {mape:.2f}%')
print(f'  MAE:              ${mae:,.0f}/month')
print(f'  RMSE:             ${rmse:,.0f}/month')
print(f'  R\u00b2:               {r2:.4f}')
print(f'  Pearson r:        {pearson_r:.4f}')
print(f'  Spearman rho:     {spearman_r:.4f}')
print(f'  Classification:   {acc:.1%}')
print(f'  Core coverage:    {core_pct:.1%}')
print(f'  Clusters found:   {n_clusters}')
print(f'  Silhouette score: {sil_score:.3f}')